# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [3]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [4]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [5]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [22]:
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')

doc = ET.parse('./data/mondial_database.xml')

In [23]:
root=doc.getroot()

In [25]:
for element in root.iterfind('country'):
    im=element.find('infant_mortality')
    if im==None:
        root.remove(element)
    #else:
    #    print im.text

In [26]:
pais=[]
infm=[]
for element in root.iterfind('country'):
    x=element.find('infant_mortality').text
    x=float(x)
    infm.append(x)
    pais.append(element.find('name').text)

In [27]:
dict={'country':pais,
     'infant_mortality':infm}

In [28]:
df=pd.DataFrame(dict)

In [29]:
df.sort_values('infant_mortality').head(10)   # The 10 countries with the lowest infant mortality rates

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [30]:
root=doc.getroot()

In [55]:
data=[]
for element in root.iterfind('country'):
    country_name=element.find('name').text
    #print country_name,
    for city in element.findall('city'):
        city_name= city.find('name').text
        #print city_name
        for population in city.iterfind('population'):
            data.append([country_name,city_name, population.attrib, population.text])
            #print country_name,city_name, population.attrib, population.text

In [70]:
df=pd.DataFrame(data,columns=['country','city','x','population'])
#df.head()

In [46]:
source=[]
year_of=[]
for i in df[2].iteritems():
    if 'measured' in i[1]:
        source.append(i[1]['measured'])
    else:
        source.append('unknown')
    year_of.append(i[1]['year'])

In [52]:
df2=df[['country','city','population']]
df2.loc[:,'data_source']=pd.Series(source, index=df2.index)
df2.loc[:,'year_of_data']=pd.Series(year_of, index=df2.index)

In [71]:
#df2.head(10)

In [69]:
                                        # Here is a sorted list of the 10 largest cities in the data, as of 1987
df3=df2.loc[df2.year_of_data=='1987']
df3.loc[:,'population']=df3.population.astype(int)  #convert the entire column 'population' from str to int
df3.sort_values('population',ascending=False).head(10)

,country,city,population,data_source,year_of_data
249,North Korea,Pyongyang,2335000,unknown,1987
154,Azerbaijan,Baku,1740000,unknown,1987
794,Algeria,Algiers,1507241,unknown,1987
17,Serbia,Beograd,1407073,unknown,1987
656,Dominican Republic,Santo Domingo,1400000,unknown,1987
70,Bulgaria,Sofia,1300000,unknown,1987
767,Uruguay,Montevideo,1247000,unknown,1987
144,Georgia,Tbilisi,1200000,unknown,1987
141,Armenia,Yerevan,1200000,unknown,1987
37,Latvia,Rīga,900000,unknown,1987


In [ ]:
root=doc.getroot()

In [ ]:
pais=[]
infm=[]
for element in root.iterfind('country'):
    x=element.find('infant_mortality').text
    x=float(x)
    infm.append(x)
    pais.append(element.find('name').text)

In [108]:
dict={'country':[],
    'population':[],
    'year':[],
    'ethnic_group':[],
    'percentage':[]}
for country in root.iterfind('country'):
    country_name=country.find('name').text
    year_list=[]
    pop_list=[]
    for population in country.iterfind('population'):
        year=population.attrib['year']
        year=int(year)
        year_list.append(year)
        pop=population.text
        pop=int(pop)
        pop_list.append(pop)
    latest=max(year_list)
    index=year_list.index(max(year_list))   #find index of first occurrence of maximum value, use in another list
    latest_pop=pop_list[index]
    
    for ethnic_group in country.iterfind('ethnicgroup'):
        percent=ethnic_group.attrib['percentage']
        propor=float(percent)/100.
        dict['country'].append(country_name)
        dict['population'].append(latest_pop)
        dict['year'].append(latest)
        dict['ethnic_group'].append(ethnic_group.text)
        dict['percentage'].append(propor)
df=pd.DataFrame(dict)       
        #print ethnic_group.text,propor#ethnic_group.attrib
        #dict[ethnic_group.text:ethnic_group.attrib['percentage']]
        #print ethnic_group.text, ethnic_group.attrib
    
    

        #city_name= city.find('name').text
        #print city_name
        #for population in city.iterfind('population'):
            #data.append([country_name,city_name, population.attrib, population.text])

In [121]:
df['eth_pop']=(df.percentage*df.population).astype('int')           #Here is list of 10 largest ethnic groups
df.eth_pop.groupby(df.ethnic_group).sum().sort_values(ascending=False).iloc[0:10]

ethnic_group
Han Chinese    1245058800
Indo-Aryan      871815583
European        494871787
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993341
Name: eth_pop, dtype: int64

In [ ]:
root=doc.getroot()

In [206]:
dict2={'c_code':[],
      'riv_len':[],
      'riv_name':[],
      'countries':[]}

for element in root.iterfind('river'):
    ctry_code=element.attrib['country']
    #print ctry_code.split()
    river_name=element.find('name').text
    located=element.find('located')
    #print country_name,
    #for river in element.findall('river'):
    if located != None:
        country_code=located.attrib['country']
        length=element.find('length')
        if length != None:
            length=int(float(length.text))
            dict2['c_code'].append(country_code)
            dict2['riv_len'].append(length)
            dict2['riv_name'].append(river_name)
            dict2['countries'].append(ctry_code.split())
        #print river_name,country_code,length
        

    
        
#        city_name= city.find('name').text
#        #print city_name
#        for population in city.iterfind('population'):
#            data.append([country_name,city_name, population.attrib, population.text])

In [208]:
df2=pd.DataFrame(dict2)
df2=df2.sort_values(by='c_code')

In [209]:
dict3={'c_code':[],
      'country':[]}
for element in root.iterfind('country'):
    dict3['country'].append(element.find('name').text)
    dict3['c_code'].append(element.attrib['car_code'])
    #print element.attrib['car_code']

In [276]:
dict3a={}
for element in root.iterfind('country'):
    dict3a[element.attrib['car_code']]=element.find('name').text

In [210]:
df3=pd.DataFrame(dict3)
df3=df3.sort_values(by='c_code')
#dict3['c_code']

In [300]:
df4=df2.merge(df3).sort_values('riv_len',ascending=False)

In [301]:
df4a=df4.head()
df4a

,c_code,countries,riv_len,riv_name,country
53,CO,"[CO, BR, PE]",6448,Amazonas,Colombia
48,CN,[CN],6380,Jangtse,China
49,CN,[CN],4845,Hwangho,China
145,R,[R],4400,Lena,Russia
204,ZRE,"[RCB, ZRE]",4374,Zaire,Zaire


In [302]:
temp=df3.country[df3.c_code=='CN']
print temp

49    China
Name: country, dtype: object


In [303]:
cntry_list=[]
for j in range(0,len(df4a.countries)):
    cstr=''
    cts=df4a.countries.iloc[j]
    for i in cts:
        cstr =cstr + ' ' + dict3a[i]
    cntry_list.append(cstr)
cntry_list

[' Colombia Brazil Peru', ' China', ' China', ' Russia', ' Congo Zaire']

In [320]:
df4a.loc[:,'countries_2']=pd.Series(cntry_list,index=df4a.index)


In [323]:
df4a.sort_values('riv_len',ascending=False)[['riv_len','riv_name','countries_2']]  #here are the longest rivers, their lengths, and at lease one of the country

,riv_len,riv_name,countries_2
53,6448,Amazonas,Colombia Brazil Peru
48,6380,Jangtse,China
49,4845,Hwangho,China
145,4400,Lena,Russia
204,4374,Zaire,Congo Zaire


In [190]:
dict5={'c_code':[],
       'ap_name':[],
       'elev':[]}

for airport in root.iterfind('airport'):
    airport_name=airport.find('name').text
    elevation=airport.find('elevation').text
    if elevation==None:
        elevation=-9999
    else:
        elevation=int(elevation)
    country_code=airport.attrib['country']
    #print elevation, country_code,airport.attrib['country']

    dict5['c_code'].append(country_code)
    dict5['elev'].append(elevation)
    dict5['ap_name'].append(airport_name)
df5=pd.DataFrame(dict5)
df6=df5.sort_values('c_code')

In [191]:
df7=df6.merge(df3)

In [193]:
df6[df6.ap_name=='Lhasa-Gonggar']

,ap_name,c_code,elev
219,Lhasa-Gonggar,CN,4005


In [194]:
df7.sort_values('elev',ascending=False).head()                   # The highest airport is in Bolivia

,ap_name,c_code,elev,country
50,El Alto Intl,BOL,4063,Bolivia
178,Lhasa-Gonggar,CN,4005,China
170,Yushu Batang,CN,3963,China
780,Juliaca,PE,3827,Peru
777,Teniente Alejandro Velasco Astete Intl,PE,3311,Peru


In [201]:
dict6={'c_code':[],
       'lake_name':[],
       'area':[]}

for lake in root.iterfind('lake'):
    lake_name=lake.find('name').text
    
    if lake.find('area')==None:
        area=-9999
    else:
        area=lake.find('area').text
        area=int(float(area))
    country_code=lake.attrib['country']
    #print elevation, country_code,airport.attrib['country']

    dict6['c_code'].append(country_code)
    dict6['area'].append(area)
    dict6['lake_name'].append(lake_name)
df8=pd.DataFrame(dict6)
df8=df8.sort_values('c_code')
df8=df8.merge(df3)

In [203]:
df8.sort_values('area',ascending=False).head()     # Here is what I get for the largest lake and its country location

,area,c_code,lake_name,country
91,57800,USA,Lake Michigan,United States
11,31792,CDN,Great Bear Lake,Canada
58,31492,R,Ozero Baikal,Russia
12,28568,CDN,Great Slave Lake,Canada
16,24420,CDN,Lake Winnipeg,Canada
